In [29]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import os 
import sys 
import json

home = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
pths = [os.path.join(home, 'astrobot')]

for pth in pths: 
    if pth not in sys.path: sys.path.insert(0, pth)

from astrobot.util import *
from astrobot.geomdataframe import GeomDataFrame 
from astrobot.r import R
from astrobot import bem_util, geom_util, mtx_util
#from astrobot.polymesh import PolyMesh

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import itertools

import ladybug_geometry.geometry3d as geom3
import ladybug_geometry.geometry2d as geom2
from ladybug_geometry_polyskel import polyskel

# Simulation
from honeybee_energy.simulation.output import SimulationOutput
from honeybee_energy.simulation.parameter import SimulationParameter
from ladybug.epw import EPW

from shapely.geometry import Polygon



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
#simdir_path = os.path.join('simulations', 'sim_0')

deeprad_model_dir = os.path.abspath(os.path.join(os.getcwd(), '../../..', 'deeprad/data/models/'))
floorplan_id = 'floorplan_1754'

gen_dir = os.path.join(deeprad_model_dir, floorplan_id)
sim_dir = os.path.join(deeprad_model_dir, floorplan_id, 'simulation')

print('gen_dir: ', gen_dir)
print('sim_dir: ', sim_dir)


gen_dir:  /mnt/c/Users/Administrator/master/git/deeprad/data/models/floorplan_1754
sim_dir:  /mnt/c/Users/Administrator/master/git/deeprad/data/models/floorplan_1754/simulation


In [31]:
# Load arrays

# TODO: automate this
num_rooms = 3
poly_np_arr = [] 
for i in range(num_rooms):
    deeprad_fpath = os.path.join(gen_dir, '{}_{}.npy'.format(floorplan_id, i))
    poly_np = np.load(deeprad_fpath)
    poly_np_arr.append(poly_np)

print(len(poly_np_arr))
#poly_np_arr

3


In [32]:


# Make Face3d with poly_arr
poly_lb_arr = []
for poly_np in poly_np_arr:
    vertices = [geom3.Point3D(*p) for p in poly_np.T[:-1]]
    poly_lb = geom3.Face3D(vertices)
    poly_lb_arr.append(poly_lb)

# Make gdf and add arrs
gdf = GeomDataFrame({'mod': [floorplan_id]})
r = R(gdf)

r.mod['mod_geom'] = [(poly_lb_arr)]
#r.mod['mod_geom'].head()

poly_lb_arr[0].vertices

(Point3D (0.03, 2.51, 0.00),
 Point3D (2.37, 2.51, 0.00),
 Point3D (2.37, 4.31, 0.00),
 Point3D (0.03, 4.31, 0.00))

In [33]:
#space_types = bem_utils.program_space_types('2013::MediumOffice')
space_types = [
     '2013::MediumOffice::ClosedOffice',
     '2013::MediumOffice::OpenOffice', 
     '2013::MediumOffice::Conference',
     '2013::MediumOffice::Corridor',
     '2013::MediumOffice::Elec/MechRoom',
     '2013::MediumOffice::Restroom']
occ = [0.67 / 3.0] * 3
oth = [0.33 / 3.0] * 3  
space_wts = np.array(occ + oth)
blended = bem_util.program_space_types_blend(space_types, space_wts, 'blended_office')

In [44]:
# Make model, assign program, constructions and wwr
mods = [0] * r.mod.shape[0]
for mod_idx in r.mod.index:
            
    # extract mod_df_row and geom
    mod_df_row = r.mod.iloc[mod_idx]
    mod_geom = mod_df_row['mod_geom']
    
    # make spaces from model geom
    spc_geoms = mod_geom  # splits floor into thermal zones
    spc_idx = range(len(spc_geoms))
    #print(spc_geoms)

    # make room2ds from spaces
    room2ds = bem_util.face3ds_to_room2ds(spc_geoms, spc_idx, mod_idx)
    [bem_util.set_room2d_program_space_type(room, blended) for room in room2ds]

    # make model from room2ds
    model = bem_util.room2ds_to_model(room2ds, id=str(mod_idx))
    # TODO: mutations like this might be more natural as OOP.
    # like optimizer(model). Can create custom model, composed of HB model, and custom setters for ml.
    # so dataframe is structure, and eigen_models/eigen_spaces are dl-hb extensions.
    bem_util.set_model_wwr(model, 0.3, None)
    bem_util.set_model_construction_set(model)
    mods[mod_idx] = model

r.mod['mod'] = mods

In [69]:
# TODO: move this into a setter w/ mod_idx, spc_geom args
# TODO: make hb_objects into lists and doc why (iterable in array)
# TODO: change list parsing to provide dataframe
# TODO: enforce check on intializing dataframes for hierarchical indexing

# define spaces
hbmods = [m.to_honeybee()[0] for m in r.mod['mod']]
spc_mod_idxs, spcs = bem_util.spcs_from_mods(hbmods, r.mod.idx)
perim_spc_mask = bem_util.perim_spcs_mask(spcs)

r.spc = GeomDataFrame(
    {'mod_idx': spc_mod_idxs[np.where(perim_spc_mask > 0)], 
     'spc_geom': [spc for p, spc in zip(perim_spc_mask, spcs) if p > 0]}
)

# define srfs
srf_mod_idxs, srf_spc_idxs, srfs = \
    bem_util.srfs_from_spcs(r.spc['spc_geom'], r.spc.idx, r.spc['mod_idx'])
perim_srf_mask = bem_util.perim_srfs_mask(srfs)

r.srf = GeomDataFrame(
    {'mod_idx': srf_mod_idxs[np.where(perim_srf_mask > 0)],
     'spc_idx': srf_spc_idxs[np.where(perim_srf_mask > 0)],
     'srf_geom': [srf for p, srf in zip(perim_srf_mask, srfs) if p > 0]}
)

# define wins
win_mod_idxs, win_spc_idxs, win_srf_idxs, wins = \
    bem_util.wins_from_srfs(r.srf['srf_geom'], r.srf.idx, r.srf.spc_idx, r.srf.mod_idx)

r.win = GeomDataFrame(
    {'mod_idx': win_mod_idxs, 
     'spc_idx': win_spc_idxs, 
     'srf_idx': win_srf_idxs,
     'win_geom': wins}
)

In [70]:
# TODO: Mtx > mtx_util, Mtx_geom > mtx_geom_util, mtx_geom, move mtx_utli to Mtx?
r.srf['type'] = [srf.type.name for srf in r.srf['srf_geom']]
r.srf['bc'] = [srf.boundary_condition.name for srf in r.srf['srf_geom']]

In [71]:
h = r.mod['mod'][0].to_honeybee()[0]
h.to_hbjson(os.path.join(sim_dir, 'test_load.hbjson'))


#print(r.srf.columns)
#r.srf.head()#[r.srf.columns].head()


'/mnt/c/Users/Administrator/master/git/deeprad/data/models/floorplan_1754/simulation/test_load.hbjson'

In [22]:
# TODO: move to bem_utils   
sim_par = SimulationParameter()
#sim_par.output.add_zone_energy_use()
#sim_par.output.add_gains_and_losses(load_type='Total')
sim_par.output.add_surface_temperature()
sim_par.output.add_surface_energy_flow()
sim_par.output.add_glazing_solar()

epw_folder, epw_file_name = os.path.split(EPW_FPATH_FOR_EXPERIMENTS)
ddy_file = os.path.join(epw_folder, epw_file_name.replace('.epw', '.ddy'))

if os.path.isfile(ddy_file):
    sim_par.sizing_parameter.add_from_ddy_996_004(ddy_file)
else:
    print('{} is not a file.'.format(ddy_file))

# write out the simulation parameters to a JSON
sim_par_dict = sim_par.to_dict()
sim_fname = 'custom_simulation_parameter.json'
sim_fpath = os.path.abspath(
    os.path.join(sim_dir, sim_fname))
with open(sim_fpath, 'w') as fp:
    json.dump(sim_par_dict, fp)

In [23]:
# Simulate
# TODO: clean up and functionalize top with sim_mods arg
sim_mods = r.mod['mod']
model_json_fnames = [0] * sim_mods.shape[0]

for i, mod in enumerate(sim_mods):
    dest_file = os.path.join(sim_dir, mod.display_name + '.json')
    model_json_fnames[i] = mod.display_name + '.json'
    with open(dest_file, 'w') as fp:
        json.dump(mod.to_dict(), fp, indent=4)
  
# Write batch files
batch_fpath = os.path.join(sim_dir, 'json_batch.sh')
epw_abs_fpath = "C:\\Users\\Administrator\\master\\git\\astrobot\\" + \
    "resources\\epw\\philadelphia\\philadelphia.epw"   
sim_abs_fpath = "C:\\Users\\Administrator\\master\\git\\deeprad\\" + \
    "data\\models\\{}\\simulation\\custom_simulation_parameter.json".format(floorplan_id)

bem_util.simulate_batch(
    batch_fpath, epw_abs_fpath, model_json_fnames, sim_abs_fpath=sim_abs_fpath)

r.mod.to_pickle(os.path.join(sim_dir, 'rmod_gen.pkl'))
r.spc.to_pickle(os.path.join(sim_dir, 'rspc_gen.pkl'))
r.srf.to_pickle(os.path.join(sim_dir, 'rsrf_gen.pkl'))
r.win.to_pickle(os.path.join(sim_dir, 'rwin_gen.pkl'))